# Create Public School Machine Learning Datasets
** This program creates all the _ML datasets in the NCEA repository.** 
* This notebook reads each School Dataset file located at \EducationDataNC\ *schoolYear* \School Datasets\ as input data.
* Different school years are processed by changing the *schoolYear* parameter.
* Different input / output files are processed / created by changing the *inputFileName* paramter in the cell below.  
* While a single program is used to create all the _ML datasets, one program copy per dataset is maintained in the repositiory so the dataset specific tranformation reports may be reviewed. 

**Datasets ending in ML are preprocessed for Machine Learning and go through the following transformations: **
1. Missing student body racial compositions are imputed using district averages.
2. Columns that have the same value in every single row are deleted.
3. Columns that have a unique value in every single row (all values are different) are deleted.
4. Empty columns (all values are NA or NULL) are deleted.
5. Numeric columns with more than the percentage of missing values specified by the *missingThreshold* parameter.
6. Remaining numeric, non-race columns with missing values are imputed / populated with 0.  In many cases, schools are not reporting values when they are zero. However, mean imputation or some other more sophisticated strategy might be considered here.
7. Categorical / text based columns with > *uniqueThreshold* unique values are deleted.
8. All remaining categorical / text based columns are one-hot encoded.  In categorical columns, one-hot encoding creates one new boolean / binary field per unique value in the target column, converting all categorical columns to a numeric data type. 
9. Duplicated or highly similar columns with > 95% correlation are delelted.    

In [2]:
#import required Libraries
import pandas as pd
import numpy as np
import os
import string

#**********************************************************************************
# Set the following variables before running this code!!!
#**********************************************************************************
#All raw data files are processed for the year below
schoolYear = 2017

#Location where copies of the raw data files will be read in from csv files.
dataDir = '../../../' + str(schoolYear) + '/School Datasets/'

#Name of the file to be processed
inputFileName = 'PublicSchools' + str(schoolYear)
#inputFileName = 'PublicHighSchools' + str(schoolYear)
#inputFileName = 'PublicMiddleSchools' + str(schoolYear)
#inputFileName = 'PublicElementarySchools' + str(schoolYear)

#Input file being transformed for machine learning 
inputFile = dataDir + inputFileName + '.csv'

#Location where the new school datasets will be created.
outputDir = '../../../' + str(schoolYear) + '/Machine Learning Datasets/'

#Missing Data Threshold (Per Column)
missingThreshold = 0.60

#Unique Value Threshold (Per Column)
#Delete Columns >  uniqueThreshold unique values prior to one-hot encoding. 
#(each unique value becomes a new column during one-hot encoding)
uniqueThreshold = 25

#Read in the School Data File
schoolData = pd.read_csv(inputFile, low_memory=False, dtype={'unit_code': object})
print('*********Start: Beginning Column and Row Counts********************************************')
schoolData.info(verbose=False)

#Select only public schools as charter schools are missing data for many columns.
schoolData = schoolData[(schoolData['type_cd'] == 'P') & (schoolData['student_num'] > 0)]

print('\r\n*********After: Selecting Only Public School Campuses**********************************')
schoolData.info(verbose=False)

#Save primary key
unit_code = schoolData['unit_code']
#Convert zip code to string
schoolData['szip_ad'] = schoolData['szip_ad'].astype('object')
#Rename Lea_Name to District Name
schoolData.rename(columns={'Lea_Name': 'District Name'}, inplace=True)

*********Start: Beginning Column and Row Counts********************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2642 entries, 0 to 2641
Columns: 2312 entries, vphone_ad to Pct of Elderly Returns
dtypes: float64(2266), int64(3), object(43)
memory usage: 46.6+ MB

*********After: Selecting Only Public School Campuses**********************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2468 entries, 2 to 2641
Columns: 2312 entries, vphone_ad to Pct of Elderly Returns
dtypes: float64(2266), int64(3), object(43)
memory usage: 43.6+ MB


# Prepare Consolidated Dataset for Machine Learning
**Below we perform operations on the entire dataset to remove columns and update row values that could cause problems during machine learning.**

## Student Body Racial Composition Features 
**Impute / update missing Student Body Racial Composition Fields using mean imputation.**
* When there are no racial composition percentages for a particular school campus / unit_code, fill in the missing values 

In [3]:
#schoolData.info(verbose=True)
#schoolData[['unit_code','School Name','_1yr_tchr_trnovr_pct','lea_1yr_tchr_trnovr_pct','st_1yr_tchr_trnovr_pct']][schoolData['_1yr_tchr_trnovr_pct']<=0.0]

In [4]:
#Get Student Body Racial Composition Fields
raceCompositionFields = schoolData.filter(regex='Indian|Asian|Hispanic|Black|White|PacificIsland|TwoOrMore|Minority')\
                                  .filter(regex='Pct').columns
    
rowsBefore = schoolData[raceCompositionFields].isnull().T.any().T.sum()

#Update missing race values with the district average when avaiable (No district averages for charter schools) 
# schoolData[raceCompositionFields] = schoolData.groupby('District Name')[raceCompositionFields]\
#                                               .transform(lambda x: x.fillna(x.mean()))

    #Review dataset contents after Racial Composition Imputation
print('*********After: Updating Missing Racial Compostion Values****************************')   
rowsAfter = schoolData[raceCompositionFields].isnull().T.any().T.sum()
rowsUpdated = rowsBefore - rowsAfter
print ('Rows Updated / Imputed: ', rowsUpdated )
print('\r\nTotal Rows Missing Racial Compositions By District Name') 
schoolData['District Name'][schoolData[raceCompositionFields].isnull().T.any().T]#.value_counts()

*********After: Updating Missing Racial Compostion Values****************************
Rows Updated / Imputed:  0

Total Rows Missing Racial Compositions By District Name


,District Name,District Name


## Remove Columns with Problematic Data
**Here we remove entire columns that could cause problems during machine learning.  The following operations are performed:**
* Remove any columns that have the same value in every single row.
* Remove any columns that have a unique value in every single row (all values are different).
* Remove empty columns (all values are NA or NULL).

In [5]:
#Remove any fields that have the same value in all rows
UniqueValueCounts = schoolData.nunique(dropna=False)
SingleValueCols = UniqueValueCounts[UniqueValueCounts == 1].index
schoolData = schoolData.drop(SingleValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with the same value in every row.*******************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(SingleValueCols))

*********After: Removing columns with the same value in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2468 entries, 2 to 2641
Columns: 2117 entries, vphone_ad to Pct of Elderly Returns
dtypes: float64(2077), object(40)
memory usage: 39.9+ MB

Columns Deleted:  195


In [6]:
#Remove any fields that have unique values in every row
schoolDataRecordCt = schoolData.shape[0]
UniqueValueCounts = schoolData.apply(pd.Series.nunique)
AllUniqueValueCols = UniqueValueCounts[UniqueValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(AllUniqueValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with unique values in every row.*******************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(AllUniqueValueCols))

*********After: Removing columns with unique values in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2468 entries, 2 to 2641
Columns: 2117 entries, vphone_ad to Pct of Elderly Returns
dtypes: float64(2077), object(40)
memory usage: 39.9+ MB

Columns Deleted:  0


In [7]:
#Remove any empty fields (null values in every row)
schoolDataRecordCt = schoolData.shape[0]
NullValueCounts = schoolData.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with null / blank values in every row.*************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with null / blank values in every row.*************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2468 entries, 2 to 2641
Columns: 2117 entries, vphone_ad to Pct of Elderly Returns
dtypes: float64(2077), object(40)
memory usage: 39.9+ MB

Columns Deleted:  0


## Handle Other Missing Values Types
* Here we eliminate any numeric columns with more than the percentage of missing values specified by the *missingThreshold* parameter.
* All remaining non-race, numeric column missing values are populated with 0.
* In many cases, it seems that schools are not simply not reporting values when they are zero. However, mean imputation or some other strategy might be considered.

In [8]:
#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean = schoolData.loc[:, (schoolData.dtypes == bool) ]
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]
sD_continuous = schoolData.loc[:, (schoolData.dtypes != bool) & (schoolData.dtypes != object)]
print( "Boolean Columns: ", sD_boolean.shape[1])
print( "Nominal Columns: ", sD_nominal.shape[1])
print( "Continuous Columns: ", sD_continuous.shape[1])
print( "Columns Accounted for: ", sD_nominal.shape[1] + sD_continuous.shape[1] + sD_boolean.shape[1])

Boolean Columns:  0
Nominal Columns:  40
Continuous Columns:  2077
Columns Accounted for:  2117


In [9]:
schoolDataRecordCt = sD_continuous.shape[0]
missingValueLimit = schoolDataRecordCt * missingThreshold
NullValueCounts = sD_continuous.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts >= missingValueLimit].index

In [10]:
NullValueCols

Index(['06_Size', '07_Size', '08_Size', '09_Size', '10_Size', '11_Size',
       '12_Size', '13_Size', 'Biology_Size', 'English II_Size',
       ...
       'HLTH_Concentrator_Ct', 'HOSP_Concentrator_Ct', 'HUMA_Concentrator_Ct',
       'INFO_Concentrator_Ct', 'LAW_Concentrator_Ct', 'MANU_Concentrator_Ct',
       'MRKT_Concentrator_Ct', 'STEM_Concentrator_Ct', 'TRAN_Concentrator_Ct',
       'Number_Industry_Recognized_Crede'],
      dtype='object', length=520)

In [8]:
#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt = sD_continuous.shape[0]
missingValueLimit = schoolDataRecordCt * missingThreshold
NullValueCounts = sD_continuous.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts >= missingValueLimit].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with >= missingThreshold % of missing values******')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with >= missingThreshold % of missing values******
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2468 entries, 2 to 2641
Columns: 1597 entries, vphone_ad to Pct of Elderly Returns
dtypes: float64(1557), object(40)
memory usage: 30.1+ MB

Columns Deleted:  520


## One-Hot Encoding of Categorical Variables
**All categorical / string variables are converted to numberic variables via one hot encoding.  Each unique row value will become a new binary / numeric column in the dataset.**
* All remaining categorical columns are one-hot encoded.  
* In categorical columns, one-hot encoding creates one new boolean / binary field per unique value in the target column, converting all categorical columns to a numeric data type. 
* Prior to one-hot encoding, columns with > *uniqueThreshold* unique values are deleted.  

In [9]:
#Delete categorical columns with > 25 unique values (Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts = schoolData[sD_nominal.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols = oneHotUniqueValueCounts[oneHotUniqueValueCounts >= uniqueThreshold].index
schoolData.drop(oneHotUniqueValueCols, axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(oneHotUniqueValueCols))

*********After: Removing columns with >= uniqueThreshold unique values***********
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2468 entries, 2 to 2641
Columns: 1580 entries, category_cd to Pct of Elderly Returns
dtypes: float64(1557), object(23)
memory usage: 29.8+ MB

Columns Deleted:  19


In [10]:
oneHotUniqueValueCols

Index(['vphone_ad', 'unit_code', 'street_ad', 'scity_ad', 'szip_ad', 'url_ad',
       'grade_range_cd', 'cover_letter_ad', 'District Name', 'District Name',
       'School_Name', 'url', 'District Name', 'District Name', 'School Name',
       'grades_BYOD', 'grades_1_to_1_access', 'SRC_Grades_Devices_Sent_Home',
       'County'],
      dtype='object')

In [11]:
#Isolate remaining categorical variables
begColumnCt = len(schoolData.columns)
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]

#one hot encode categorical variables
schoolData = pd.get_dummies(data=schoolData, 
                       columns=sD_nominal.columns, drop_first=True)

#Determine change in column count
endColumnCt = len(schoolData.columns)
columnsAdded = endColumnCt - begColumnCt

#Review dataset contents one hot high unique value drops
print ('Columns To One-Hot Encode: ', len(sD_nominal.columns))
print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
schoolData.info(verbose=False)
print ('\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)

Columns To One-Hot Encode:  23

*********After: Adding New Columns Via One-Hot Encoding*************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2468 entries, 2 to 2641
Columns: 1616 entries, student_num to SRC_devices_sent_home_Yes
dtypes: float64(1557), uint8(59)
memory usage: 29.5 MB

New Columns Created Via One-Hot Encoding:  36


## Impute any Remaining Missing Values as Zero

In [12]:
#Print out all the missing value rows
pd.set_option('display.max_rows', 1000)

print('\r\n*********The Remaining Missing Values Below will be set to Zero!*************************')

#Check for Missing values 
missing_values = schoolData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values




*********The Remaining Missing Values Below will be set to Zero!*************************


,Variable Name,Number Missing Values
3,00_Size,1153
4,01_Size,1154
5,02_Size,1152
6,03_Size,1159
7,04_Size,1149
8,05_Size,1180
9,Math I_Size,1428
18,SPG Score,131
19,Reading SPG Score,622
20,Math SPG Score,622


In [13]:
#Replace all remaining NaN with 0
schoolData = schoolData.fillna(0)

#Check for Missing values after final imputation 
missing_values = schoolData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values

,Variable Name,Number Missing Values


## Identify and Remove Highly Correlated Features
**Find and remove any columns / features that are > 95% correlated**
* https://stackoverflow.com/questions/39409866/correlation-heatmap
* https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
* https://codeyarns.com/2015/04/20/how-to-change-font-size-in-seaborn/

In [14]:
# calculate the correlation matrix
corr_matrix  = schoolData.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [15]:
#Get all of the correlation values > 95%
x = np.where(upper > 0.95)

#Display all field combinations with > 95% correlation
cf = pd.DataFrame()
cf['Field1'] = upper.columns[x[1]]
cf['Field2'] = upper.index[x[0]]

#Get the correlation values for every field combination. (There must be a more pythonic way to do this!)
corr = [0] * len(cf)
for i in range(0, len(cf)):
    corr[i] =  upper[cf['Field1'][i]][cf['Field2'][i]] 
    
cf['Correlation'] = corr

print ('There are ', str(len(cf['Field1'])), ' field correlations > 95%.')
cf

There are  282040  field correlations > 95%.


,Field1,Field2,Correlation
0,class_teach_num,student_num,0.967326
1,lea_class_teach_num,lea_avg_student_num,0.969942
2,st_class_teach_num,st_avg_student_num,0.999094
3,st_advance_dgr_pct,st_avg_student_num,0.985283
4,01_Size,00_Size,0.952367
5,Overall Achievement Score,SPG Score,0.987985
6,Math SPG Score,Reading SPG Score,0.975399
7,Reading Score,Reading SPG Score,0.993523
8,Math Score,Reading SPG Score,0.962399
9,EOGMathGr3-8_GLP_All,Reading SPG Score,0.956264


In [16]:
print ('Dropping the following ', str(len(to_drop)), ' highly correlated fields.')
to_drop

Dropping the following  1210  highly correlated fields.


['01_Size',
 'Math SPG Score',
 'Overall Achievement Score',
 'Reading Score',
 'Math Score',
 'EOCMathI_CACR_All',
 'EOCMathI_GLP_All',
 'EOCSubjects_CACR_All',
 'EOCSubjects_GLP_All',
 'EOG/EOCSubjects_GLP_All',
 'EOGGr3_GLP_All',
 'EOGGr4_GLP_All',
 'EOGGr5_GLP_All',
 'EOGMathGr3_CACR_All',
 'EOGMathGr3_GLP_All',
 'EOGMathGr3-8_CACR_All',
 'EOGMathGr3-8_GLP_All',
 'EOGMathGr4_CACR_All',
 'EOGMathGr4_GLP_All',
 'EOGMathGr5_CACR_All',
 'EOGMathGr5_GLP_All',
 'EOGReadingGr3_CACR_All',
 'EOGReadingGr3_GLP_All',
 'EOGReadingGr3-8_CACR_All',
 'EOGReadingGr3-8_GLP_All',
 'EOGReadingGr4_CACR_All',
 'EOGReadingGr4_GLP_All',
 'EOGReadingGr5_CACR_All',
 'EOGReadingGr5_GLP_All',
 'EOGScienceGr5_CACR_All',
 'EOGScienceGr5_GLP_All',
 'EOGScienceGr5&8_CACR_All',
 'EOGScienceGr5&8_GLP_All',
 'EOGSubjects_CACR_All',
 'EOGSubjects_GLP_All',
 'EOCMathI_GLP_Female',
 'EOCSubjects_CACR_Female',
 'EOCSubjects_GLP_Female',
 'EOG/EOCSubjects_CACR_Female',
 'EOG/EOCSubjects_GLP_Female',
 'EOGGr3_CACR_Female

In [17]:
#Check columns before drop 
print('\r\n*********Before: Dropping Highly Correlated Fields*************************************')
schoolData.info(verbose=False)

# Drop the highly correlated features from our training data 
schoolData = schoolData.drop(to_drop, axis=1)

#Check columns after drop 
print('\r\n*********After: Dropping Highly Correlated Fields**************************************')
schoolData.info(verbose=False)


*********Before: Dropping Highly Correlated Fields*************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2468 entries, 2 to 2641
Columns: 1616 entries, student_num to SRC_devices_sent_home_Yes
dtypes: float64(1557), uint8(59)
memory usage: 29.5 MB

*********After: Dropping Highly Correlated Fields**************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2468 entries, 2 to 2641
Columns: 406 entries, student_num to SRC_devices_sent_home_Yes
dtypes: float64(363), uint8(43)
memory usage: 7.0 MB


In [18]:
#Restore the unit_code before saving
schoolData['unit_code'] = unit_code
#Save the final dataset to a .csv file
schoolData.to_csv(outputDir + inputFileName + '_ML.csv', sep=',', index=False)

In [19]:
print('*********FINAL DATASET DETAILS*********************************************************\r\n')
schoolData.info(verbose=True)

*********FINAL DATASET DETAILS*********************************************************

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2468 entries, 2 to 2641
Data columns (total 407 columns):
student_num                                                     float64
lea_avg_student_num                                             float64
st_avg_student_num                                              float64
00_Size                                                         float64
02_Size                                                         float64
03_Size                                                         float64
04_Size                                                         float64
05_Size                                                         float64
Math I_Size                                                     float64
lea_total_expense_num                                           float64
lea_salary_expense_pct                                          float64
lea_service

In [20]:
import sklearn
import pandas as pd

print('Sklearn Version: ' + sklearn.__version__)
print('Pandas Version: ' + pd.__version__)

Sklearn Version: 0.21.2
Pandas Version: 0.24.2


In [21]:
print ('Output File Location:\r\n\r\n' + outputDir + inputFileName + '_ML.csv')

Output File Location:

../../../2017/Machine Learning Datasets/PublicSchools2017_ML.csv
